## Actividad práctica: Predicción de calidad de aire

Descargue e importe el siguiente dataset usando pandas

    https://archive.ics.uci.edu/ml/datasets/Air+Quality

El objetivo de esta actividad es entrenar un modelo para predecir 
- la concentración de Monóxido de Carbono (CO)
- la concentración de Benzeno (C6H6)

> Observe como se comportan los datos, en base a esto proponga un modelo lineal en sus parámetros

Para entrenar su predictor use
- los primeros dos meses de datos para ajustar su predictor
- el tercer mes de datos para validar sus parámetros 


Nota: Un valor de "-200" corresponde a un *missing value*

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Importar datos

In [ ]:
!wget -c https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip
!unzip AirQualityUCI.zip

df = pd.read_csv("AirQualityUCI.csv", sep=';', decimal=',')
df = df[df.columns[:-2]] 
df.dropna(inplace=True)
df["Date_time"] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format="%d/%m/%Y %H.%M.%S")
df.drop(["Date", "Time"], axis=1, inplace=True)
df.head(n=5)
df.set_index("Date_time", inplace=True)

!rm AirQualityUCI*

Visualizar primeros dos meses para la variable CO

Existe un comportamiento periódico diario y semanal

Un modelo sinusoidal podría ser apropiado

In [ ]:
sub_df = df[df["CO(GT)"] > -200].loc[:pd.to_datetime("06/10/2004 18:00:00")]["CO(GT)"]
time, data = sub_df.index, sub_df.values

fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)
ax.plot(time, data)
fig.autofmt_xdate()

In [ ]:
data_float = data.astype('float32')
time_float = np.array([t.timestamp()/(24*3600) - time[0].timestamp()/(24*3600) for t in time])

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(6, 4), tight_layout=True)
mask = time_float <= 60 # Primer mes
ax[0].plot(time_float[mask], data[mask], label='data')

def build_phi(time, freqs=[*range(10), 1/3.5, 1/7, 1/14]):
    phi_cos = np.stack([np.cos(2*np.pi*time*f) for f in [0]+freqs])
    phi_sin = np.stack([np.sin(2*np.pi*time*f) for f in freqs])
    return np.concatenate((phi_cos, phi_sin), axis=0).T

X_lstsq = build_phi(time_float[mask])
theta, MSE, rank, singval = np.linalg.lstsq(X_lstsq, data[mask], rcond=None)

ax[0].plot(time_float[mask], np.dot(X_lstsq, theta), label='modelo')
ax[0].legend()
ax[1].plot(time_float[mask], np.abs(data[mask] - np.dot(X_lstsq, theta)))
ax[1].set_ylabel('Error absoluto');

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(6, 3), tight_layout=True)
mask = time_float > 60
ax[0].plot(time_float[mask], data[mask])
X_lstsq = build_phi(time_float[mask])
mse = np.mean(np.power(np.dot(X_lstsq, theta) - data[mask], 2))
display(f"Error cuadrático medio {mse}")
ax[0].plot(time_float[mask], np.dot(X_lstsq, theta));

x_smooth = np.linspace(60, 90, num=1000)
X_lstsq = build_phi(x_smooth)
ax[1].plot(x_smooth, np.dot(X_lstsq, theta));